In [25]:
import pandas as pd
import numpy as np
file_name = r'C:\Users\year3\Downloads\Lab Session Data.xlsx'
sheet_name = 'thyroid0387_UCI'
df = pd.read_excel(file_name , sheet_name=sheet_name)
print("dataframe:")
print(df)

   # Infer data types
types = infer_data_types(df)
    
    # Print inferred data types
print("Inferred data types:")
for column, dtype in types.items():
    print(f"{column}: {dtype}")

dataframe:
      Record ID  age sex on thyroxine query on thyroxine  \
0     840801013   29   F            f                  f   
1     840801014   29   F            f                  f   
2     840801042   41   F            f                  f   
3     840803046   36   F            f                  f   
4     840803047   32   F            f                  f   
...         ...  ...  ..          ...                ...   
9167  870119022   56   M            f                  f   
9168  870119023   22   M            f                  f   
9169  870119025   69   M            f                  f   
9170  870119027   47   F            f                  f   
9171  870119035   31   M            f                  f   

     on antithyroid medication sick pregnant thyroid surgery I131 treatment  \
0                            f    f        f               f              f   
1                            f    f        f               f              f   
2                            f 

In [26]:
def infer_data_types(df):
    data_types = {}
    
    for column in df.columns:
        # Check for numeric data
        if pd.api.types.is_numeric_dtype(df[column]):
            data_types[column] = 'numeric'
        
        # Check for boolean data
        elif df[column].dropna().apply(lambda x: x in ['t', 'f', 'True', 'False']).all():
            data_types[column] = 'boolean'
        
        # Check for categorical/string data
        elif df[column].dtype == object:
            # Check if it has a lot of unique values; if not, treat as categorical
            if len(df[column].unique()) < 10:
                data_types[column] = 'categorical'
            else:
                data_types[column] = 'string'
                
        # Check for dates
        elif pd.api.types.is_datetime64_any_dtype(df[column]):
            data_types[column] = 'datetime'
            
        # Default to string if no other type matches
        else:
            data_types[column] = 'string'
    
    return data_types

In [14]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

# Define columns to be One-Hot Encoded
one_hot_columns = [
    'sex', 'on thyroxine', 'query on thyroxine', 'on antithyroid medication', 
    'sick', 'pregnant', 'thyroid surgery', 'I131 treatment', 
    'query hypothyroid', 'query hyperthyroid', 'lithium', 'goitre', 
    'tumor', 'hypopituitary', 'psych', 'TSH measured', 'T3 measured', 
    'TT4 measured', 'T4U measured', 'FTI measured', 'TBG measured', 
    'referral source', 'Condition'
]

# One-Hot Encoding
df_encoded = pd.get_dummies(df, columns=one_hot_columns)

# Label Encoding for ordinal data (if needed)
# Here 'age' could be categorized if you need to use Label Encoding
# For demonstration, let's assume we categorize age into groups
df['age_group'] = pd.cut(df['age'], bins=[20, 30, 40, 50, 60, 70, 80], labels=['20-30', '31-40', '41-50', '51-60', '61-70', '71-80'])
label_encoder = LabelEncoder()
df['age_group_encoded'] = label_encoder.fit_transform(df['age_group'])

# Drop original categorical columns if not needed
df_encoded = df_encoded.drop(columns=['age'])

print(df_encoded.head())

   Record ID  TSH   T3  TT4 T4U FTI TBG  sex_?  sex_F  sex_M  ...  \
0  840801013  0.3    ?    ?   ?   ?   ?      0      1      0  ...   
1  840801014  1.6  1.9  128   ?   ?   ?      0      1      0  ...   
2  840801042    ?    ?    ?   ?   ?  11      0      1      0  ...   
3  840803046    ?    ?    ?   ?   ?  26      0      1      0  ...   
4  840803047    ?    ?    ?   ?   ?  36      0      1      0  ...   

   Condition_MI  Condition_MK  Condition_N  Condition_NO CONDITION  \
0             0             0            0                       1   
1             0             0            0                       1   
2             0             0            0                       1   
3             0             0            0                       1   
4             0             0            0                       0   

   Condition_O  Condition_OI  Condition_P  Condition_Q  Condition_R  \
0            0             0            0            0            0   
1            0        

In [15]:
# Convert numeric columns to appropriate types
df['TSH'] = pd.to_numeric(df['TSH'], errors='coerce')
df['T3'] = pd.to_numeric(df['T3'], errors='coerce')
df['TT4'] = pd.to_numeric(df['TT4'], errors='coerce')
df['T4U'] = pd.to_numeric(df['T4U'], errors='coerce')
df['FTI'] = pd.to_numeric(df['FTI'], errors='coerce')
df['TBG'] = pd.to_numeric(df['TBG'], errors='coerce')

# Compute range for numeric variables
numeric_columns = ['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'TBG']
range_summary = {}

for column in numeric_columns:
    min_value = df[column].min()
    max_value = df[column].max()
    range_summary[column] = (min_value, max_value)

# Display the range for each numeric column
for column, (min_val, max_val) in range_summary.items():
    print(f"{column} - Min: {min_val}, Max: {max_val}")

age - Min: 1, Max: 65526
TSH - Min: 0.005, Max: 530.0
T3 - Min: 0.05, Max: 18.0
TT4 - Min: 2.0, Max: 600.0
T4U - Min: 0.17, Max: 2.33
FTI - Min: 1.4, Max: 881.0
TBG - Min: 0.1, Max: 200.0


In [16]:
# Check for missing values
missing_values = df.isna().sum()

# Print missing values
print("Missing values in each attribute:")
print(missing_values)

# For completeness, also check if there are any 'None' or '?' values that may need to be treated as missing
non_numeric_columns = df.select_dtypes(include=['object']).columns

for column in non_numeric_columns:
    unique_values = df[column].unique()
    print(f"\nUnique values in column '{column}':")
    print(unique_values)

Missing values in each attribute:
Record ID                       0
age                             0
sex                             0
on thyroxine                    0
query on thyroxine              0
on antithyroid medication       0
sick                            0
pregnant                        0
thyroid surgery                 0
I131 treatment                  0
query hypothyroid               0
query hyperthyroid              0
lithium                         0
goitre                          0
tumor                           0
hypopituitary                   0
psych                           0
TSH measured                    0
TSH                           842
T3 measured                     0
T3                           2604
TT4 measured                    0
TT4                           442
T4U measured                    0
T4U                           809
FTI measured                    0
FTI                           802
TBG measured                    0
TBG           

In [27]:
# Fill missing values with the median of each column for simplicity
df.fillna(df.median(numeric_only=True), inplace=True)
def detect_outliers_iqr(df, column):
    """ Detect outliers using the IQR method. """
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    outliers = df[(df[column] < (Q1 - 1.5 * IQR)) | (df[column] > (Q3 + 1.5 * IQR))]
    return outliers

# Identify numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Detect outliers for each numeric column
outlier_results = {}
for col in numeric_columns:
    outliers = detect_outliers_iqr(df, col)
    if not outliers.empty:
        outlier_results[col] = outliers

# Print outliers
for col, outliers in outlier_results.items():
    print(f"Outliers in column '{col}':")
    print(outliers)
    print("\n")

Outliers in column 'age':
      Record ID    age sex on thyroxine query on thyroxine  \
2976  850530001    455   F            f                  f   
5710  860210008  65511   M            f                  f   
6392  860403050  65512   M            f                  f   
8105  861014041  65526   F            f                  f   

     on antithyroid medication sick pregnant thyroid surgery I131 treatment  \
2976                         f    f        f               f              f   
5710                         f    f        f               f              f   
6392                         f    f        f               f              f   
8105                         f    f        f               f              f   

      ... TT4 measured  TT4 T4U measured   T4U FTI measured  FTI TBG measured  \
2976  ...            t  118            t  1.13            t  104            f   
5710  ...            t  113            t  1.08            t  104            f   
6392  ...            t  

In [28]:
# Calculate mean and variance/standard deviation for numeric columns
numeric_columns = df.select_dtypes(include=[np.number]).columns

# Create a dictionary to store the results
stats_results = {}

for col in numeric_columns:
    mean = df[col].mean()
    variance = df[col].var()
    std_dev = df[col].std()
    stats_results[col] = {'mean': mean, 'variance': variance, 'standard deviation': std_dev}

# Print the results
for col, stats in stats_results.items():
    print(f"Statistics for column '{col}':")
    print(f"  Mean: {stats['mean']}")
    print(f"  Variance: {stats['variance']}")
    print(f"  Standard Deviation: {stats['standard deviation']}")
    print("\n")

Statistics for column 'Record ID':
  Mean: 852947346.6122983
  Variance: 57486250586150.34
  Standard Deviation: 7581968.780346589


Statistics for column 'age':
  Mean: 73.55582206716092
  Variance: 1401800.8688713463
  Standard Deviation: 1183.9767180444667


